# Анализ характеристик

- Я хочу выбрать для анализа DIST-граф, в первой части он оказался более эффективен. Первым делом переберу все характеристики и оценю их мощность.

In [37]:
import sys
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join('..', 'src')))

import importlib
import functions
importlib.reload(functions)

<module 'functions' from '/Users/berdov/dm_pr/dm/src/functions.py'>

In [32]:
from functions import chromatic_number
from functions import clique_number
from functions import max_independent_set_size
from functions import domination_number
from functions import clique_cover_number

In [33]:
from functions import (
    build_dist_graph,
    sample_gamma,
    sample_exp,
    monte_carlo_characteristic
)

- теперь мы готовы к анализу характеристик и оценке того, как они меняются с ростом n. В качестве d возьмем 0.1, он показал себя лучше остальных в 1 части эксперимента.

In [36]:
n = [10, 25, 100, 200, 500]
d = 0.1
n_sim = 200
alpha = 0.05

characteristics = {
    'Хроматическое число': chromatic_number,
    'Кликовое число': clique_number,
    'Макс. независимое множество': max_independent_set_size,
    'Доминирование': domination_number,
    'Кликовое покрытие': clique_cover_number
}

summary = []

for n in n:
    for name, func in characteristics.items():
        print(f"Анализ: {name}")
        lambda_H0 = 0.5**0.5
        lambda_H1 = 1
        X0_raw = monte_carlo_characteristic(sample_gamma, lambda X: build_dist_graph(X, d), func, 0.5, lambda_H0, n=n, n_sim=n_sim)
        X1_raw = monte_carlo_characteristic(sample_exp, lambda X: build_dist_graph(X, d), func, lambda_H1, n=n, n_sim=n_sim)
        if np.median(X0_raw) < np.median(X1_raw):
            T_H0, T_H1 = X0_raw, X1_raw
        else:
            T_H0, T_H1 = X1_raw, X0_raw

        threshold = np.percentile(T_H0, 100 * (1 - alpha))
        power = np.mean(T_H1 > threshold)
        auc = roc_auc_score([0]*len(T_H0) + [1]*len(T_H1), np.concatenate([T_H0, T_H1]))
        summary.append({
            'n': n,
            'Характеристика': name,
            'AUC ROC': round(auc, 3),
            'Power (H1)': round(power, 3),
            'Порог (95% H0)': round(threshold, 2)
        })

df_summary = pd.DataFrame(summary)
print(df_summary)

Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие
      n               Характеристика  AUC ROC  Power (H1)  Порог (95% H0)
0    10          Хроматическое число    0.705       0.155            4.00
1    10               Кликовое число    0.744       0.200            4.00
2    10  Макс. независимое множество    0.723       0.105            8.00
3    10                Домини

# Вывод при росте n

- заметим, что каждый из критериев повышал свою эффективность при росте n, кроме размера максимального независимого множества. Удивительно, но его мощность лишь падает. Заметим что при n=500/200 у нас есть аж три критерия с мощностью 1. Значит мы уж точно справимся с построением качественных моделей


- теперь стало понятно как строить модель. Давайте сравним 4 модели и выберем лучшую из них, также будем анализировать результаты при разных N.

In [44]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd

models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM (RBF)": SVC(probability=True),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
}

n_sim = 200
d = 0.1
n_values = [25, 100, 500]

feature_funcs = [
    max_independent_set_size,
    domination_number,
    clique_number,
    chromatic_number,
    clique_cover_number
]

results = []

for n in n_values:
    df_train = generate_dataset(
        n=n,
        d=d,
        dist_H0=sample_gamma,
        args_H0=(0.5, 0.5**0.5),
        dist_H1=sample_exp,
        args_H1=(1,),
        feature_funcs=feature_funcs,
        graph_func=lambda X: build_dist_graph(X, d),
        n_sim=n_sim
    )

    X_train = df_train.drop("label", axis=1)
    y_train = df_train["label"]

    df_H0 = generate_dataset(n=n, d=d, dist_H0=sample_gamma, args_H0=(0.5, 0.5**0.5),
                             dist_H1=sample_exp, args_H1=(1,),
                             feature_funcs=feature_funcs,
                             graph_func=lambda X: build_dist_graph(X, d),
                             n_sim=100).query("label == 0")

    df_H1 = generate_dataset(n=n, d=d, dist_H0=sample_gamma, args_H0=(0.5, 0.5**0.5),
                             dist_H1=sample_exp, args_H1=(1,),
                             feature_funcs=feature_funcs,
                             graph_func=lambda X: build_dist_graph(X, d),
                             n_sim=100).query("label == 1")

    for name, model in models.items():
        auc_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
        acc_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

        clf = model.__class__(**model.get_params())
        clf.fit(X_train, y_train)

        y_pred_H0 = clf.predict(df_H0.drop("label", axis=1))
        y_pred_H1 = clf.predict(df_H1.drop("label", axis=1))

        type1 = np.mean(y_pred_H0 == 1)
        power = np.mean(y_pred_H1 == 1)

        results.append({
            "n": n,
            "Модель": name,
            "AUC Mean": round(np.mean(auc_scores), 3),
            "AUC Std": round(np.std(auc_scores), 3),
            "Accuracy Mean": round(np.mean(acc_scores), 3),
            "Accuracy Std": round(np.std(acc_scores), 3),
            "Power": round(power, 3),
            "Type I Error": round(type1, 3)
        })

df_results = pd.DataFrame(results)
print(df_results)

      n               Модель  AUC Mean  AUC Std  Accuracy Mean  Accuracy Std  \
0    25        Random Forest     0.905    0.027          0.817         0.050   
1    25  Logistic Regression     0.930    0.022          0.845         0.030   
2    25            SVM (RBF)     0.929    0.024          0.847         0.041   
3    25        Decision Tree     0.788    0.052          0.778         0.040   
4   100        Random Forest     0.997    0.005          0.995         0.006   
5   100  Logistic Regression     1.000    0.000          0.995         0.006   
6   100            SVM (RBF)     1.000    0.000          0.995         0.006   
7   100        Decision Tree     0.988    0.014          0.988         0.014   
8   500        Random Forest     1.000    0.000          1.000         0.000   
9   500  Logistic Regression     1.000    0.000          1.000         0.000   
10  500            SVM (RBF)     1.000    0.000          1.000         0.000   
11  500        Decision Tree     1.000  

# Выводы

- для начала выберем лучшую модель. При n = 500 все модели отработали просто идеально, все показатели равны твердой единичке. Заметим, при n = 100 также power=1, при этом верояность ошибки стала 0.01 у всех 4х моделей, LogReg и SVM отработали чуть лучше, у них дисперсия по AUC = 0 и наименьшая по критерию Accuracy.  При n = 25 все модели отработали хуже, но это проблематика размера выборки. LogReg и SVM в очередной раз справились чуть лучше. В качестве победителя выявим LogReg, у нее самая высокая мощность (0.81) и самая маленкьая дисперсия Accuracy (0.03). К сожалению вероятность ошибки 1 рода = 0.21, но все равно результаты просто прекрасные.

# Главный вывод: все модели справились очень хорошо